# Лабораторная работа №4

## Задание 1 . 
###### Провести классификацию найденного датасета, методами наивного Байеса  . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

датасет: [Homicide Reports, 1980-2014](https://www.kaggle.com/datasets/murderaccountability/homicide-reports)

### Загрузка Библиотек

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

### Загрузка данных и предварительная обработка
###### Загрузка данных

In [2]:
data = pd.read_csv('database.csv',low_memory=False)
# Удаление ненужных столбцов
data = data.drop(columns=["Record ID", "Record Source"])
# Заполнение пропущенных значений
data = data.dropna()

###### Выбираем случайное подмножество данных, т.к. загруженный датасет содержит 638 454 записей которые будут обрабатываться слишком долго

In [3]:
subset_data = data.sample(n=11000, random_state=42)

subset_data.head() # Просмотр первых нескольких строк данных

,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,Crime Solved,...,Victim Race,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count
484984,CA03612,Victorville,Municipal Police,San Bernardino,California,2005,July,1,Murder or Manslaughter,No,...,White,Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,1,0
608445,TX10100,Harris County,Sheriff,Harris,Texas,2013,May,3,Murder or Manslaughter,No,...,White,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Firearm,0,0
417612,AR06200,St. Francis,Sheriff,St. Francis,Arkansas,2001,June,1,Murder or Manslaughter,Yes,...,Black,Not Hispanic,Male,18,Black,Not Hispanic,Acquaintance,Handgun,0,0
372692,AZ01003,Tucson,Municipal Police,Pima,Arizona,1998,March,6,Murder or Manslaughter,No,...,White,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,0,0
402028,WA02400,Okanogan,Sheriff,Okanogan,Washington,1999,August,1,Murder or Manslaughter,Yes,...,White,Unknown,Male,20,White,Unknown,Girlfriend,Rifle,0,0


###### Выбор признаков и целевой переменной

In [20]:
X = subset_data[['Victim Sex', 'Victim Age', 'Victim Race', 'Perpetrator Sex', 'Perpetrator Age', 'Perpetrator Race', 'Weapon']]
Y = subset_data['Crime Solved']
# можно заменить data на subset_data и наоборот

###### Преобразование категориальных признаков в числовые

In [9]:
X = pd.get_dummies(X)

###### Разделение данных на обучающий и тестовый наборы

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

###### Стандартизация данных

In [12]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

###### Настройка гиперпараметров с использованием Grid Search

In [19]:
param_grid = {'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3]}
grid_search = GridSearchCV(GaussianNB(), param_grid, cv=5)
grid_search.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=GaussianNB(),
             param_grid={'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05,
                                           0.0001, 0.001]})

###### Лучшая модель

In [21]:
best_params = grid_search.best_params_
print("Best var_smoothing:", best_params)

Best var_smoothing: {'var_smoothing': 1e-09}


###### Вывод результатов кросс-валидации

In [25]:
cv_results = pd.DataFrame(grid_search.cv_results_)
print(cv_results[['param_var_smoothing', 'mean_test_score', 'std_test_score']])

  param_var_smoothing  mean_test_score  std_test_score
0                 0.0         0.998835        0.000095
1                 0.0         0.998835        0.000095
2                 0.0         0.998817        0.000097
3            0.000001         0.998817        0.000097
4             0.00001         0.998817        0.000103
5              0.0001         0.998806        0.000111
6               0.001         0.998770        0.000109


###### Обучение модели с лучшими гиперпараметрами

In [26]:
best_var_smoothing = best_params['var_smoothing']
model = GaussianNB(var_smoothing=best_var_smoothing)
model.fit(X_train, Y_train)

GaussianNB()

###### Оценка модели на тестовом наборе

In [27]:
Y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

Accuracy: 0.9989505916626857
              precision    recall  f1-score   support

          No       1.00      1.00      1.00     37875
         Yes       1.00      1.00      1.00     89816

    accuracy                           1.00    127691
   macro avg       1.00      1.00      1.00    127691
weighted avg       1.00      1.00      1.00    127691

